In [ ]:
%autosave 15
%matplotlib inline

import numpy as np
import scipy as sp
import pandas as pd
import math as math
import random
import os

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [ ]:
SPAM = "spmsg"
LEGIT = "legit"

class Email:
    def __init__(self, filename, lines):
        self.filename = filename
        self.isSpam = SPAM in filename
        
        self.title = list(map(int, (lines[0].strip().split())[1:]))
        self.body = []
        for line in lines[2:]:
            self.body += list(map(int, line.strip().split()))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        kind = "LEGIT"
        if (self.isSpam):
            kind = "SPAM"
        return kind + " EMAIL " + self.filename + ": " + str(self.title) + ", " + str(self.body) + "\n"

In [ ]:
class DictValue:
    def __init__(self):
        self.sTitle = 0
        self.lTitle = 0
        self.sBody = 0
        self.lBody = 0
        
    def foundInTitle(self, isSpam):
        if (isSpam):
            self.sTitle += 1
        else:
            self.lTitle += 1
            
    def foundInBody(self, isSpam):
        if (isSpam):
            self.sBody += 1
        else:
            self.lBody += 1
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        t = (self.sTitle, self.lTitle)
        b = (self.sBody, self.lBody)
        return "TITLE: " + str(t) + ", BODY: " + str(b)

In [ ]:
PARTS_DIR = "./Bayes/pu1/"
PARTS_NUM = 10

def getData():
    parts = []  
    for i in range(PARTS_NUM):
        curPart = PARTS_DIR + "part" + str(i + 1)
        emails = []
        for filename in os.listdir(curPart):
            file = open(curPart + "/" + filename, 'r')
            emails.append(Email(filename, file.readlines()))
            file.close()
        parts.append(emails)
    return parts

parts = getData()

In [ ]:
# takes: array of emails
# returns: Dict where keys are words and values are DictValues (contain key-word statistic for all given emails)
def learnBayes(emails):
    dictName = {}
    counters = [0, 0]
    unique = set()
    for email in emails:
        for word in email.title:
            unique.add(word)
            dictName.setdefault(word, DictValue()).foundInTitle(email.isSpam)
            if email.isSpam:
                counters[0] += 1
            else:
                counters[1] += 1
        for word in email.body:
            unique.add(word)
            dictName.setdefault(word, DictValue()).foundInBody(email.isSpam)
            if email.isSpam:
                counters[0] += 1
            else:
                counters[1] += 1

    return dictName, counters, len(unique)
            
#print(learnBayes(parts[0]))

In [ ]:
def testBayes(spamDict, counters, emails, ns, n, q, z):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    min_diff = 10000000000
    max_diff = -10000000000
    for email in emails:
        p_spam = math.log(ns / n)
        p_legit = math.log((n - ns) / n)
        
        for word in email.title:
            wf = spamDict.get(word, DictValue())
            p_spam += math.log((wf.sTitle + wf.sBody + z) / (counters[0] + z * q))
            p_legit += math.log((wf.lTitle + wf.lBody + z) / (counters[1] + z * q))

        for word in email.body:
            wf = spamDict.get(word, DictValue())
            p_spam += math.log((wf.sTitle + wf.sBody + z) / (counters[0] + q * z))
            p_legit += math.log((wf.lTitle + wf.lBody + z) / (counters[1] + q * z))

        isCurrentSpam = p_spam > p_legit
        if (email.isSpam != isCurrentSpam):
            if isCurrentSpam:
                FP += 1
            else:
                FN += 1
        else:
            if isCurrentSpam:
                TP += 1
            else:
                TN += 1
        
        if (p_spam - p_legit > max_diff):
            max_diff = p_spam - p_legit
        if (p_spam - p_legit < min_diff):
            min_diff = p_spam - p_legit
    if TP == 0:
        return 0
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    print(FP)

    return 2 * (precision * recall) / (precision + recall), min_diff, max_diff

In [ ]:
def cv():
    best_result = None
    best_score  = -1

    min_best = 10000000000
    max_best = -10000000000
    aver = 0
    for i in range(PARTS_NUM):
        learningSet = []
        for j in range(PARTS_NUM):
            if (j != i):
                learningSet += parts[j]
                
        ns = 0
        n = len(learningSet)
        for email in learningSet:
            if (email.isSpam):
                ns += 1
                
        testingSet  = parts[i]
        
        
        spamDict, counters, unique = learnBayes(learningSet)
        score, min_diff, max_diff = testBayes(spamDict, counters, testingSet, ns, n, unique, 0.01706422018348624)

        if score > best_score:
            min_best = min_diff
            max_best = max_diff
            best_score = score
            best_result = spamDict, counters, testingSet, ns, n, unique
        print(score)
        aver += score
    print("Average:", aver / PARTS_NUM)
    
    return best_result, min_best, max_best

In [ ]:
result, min_v, max_v = cv()
spamDict, counters, testingSet, ns, n, unique = result

In [ ]:
def getF1Score(res, diff):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for (d, p_spam, p_legit, isSpam) in res:
        isCurrentSpam = d > diff
        if (isSpam != isCurrentSpam):
            if isCurrentSpam:
                FP += 1
            else:
                FN += 1
        else:
            if isCurrentSpam:
                TP += 1
            else:
                TN += 1
                
    if TP == 0:
        return 0
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    
    #print("FP", FP)

    return 2 * (precision * recall) / (precision + recall)



def testBayes2(spamDict, counters, emails, ns, n, q, z):
    res = []
    for email in emails:
        p_spam = math.log(ns / n)
        p_legit = math.log((n - ns) / n)
        
        for word in email.title:
            wf = spamDict.get(word, DictValue())
            p_spam += math.log((wf.sTitle + wf.sBody + z) / (counters[0] + z * q))
            p_legit += math.log((wf.lTitle + wf.lBody + z) / (counters[1] + z * q))

        for word in email.body:
            wf = spamDict.get(word, DictValue())
            p_spam += math.log((wf.sTitle + wf.sBody + z) / (counters[0] + q * z))
            p_legit += math.log((wf.lTitle + wf.lBody + z) / (counters[1] + q * z))
    
        res.append((p_spam - p_legit, p_spam, p_legit, email.isSpam))
    return res

In [ ]:
def cv2():
    best_result = None
    best_score  = -1

    for i in range(PARTS_NUM):
        learningSet = []
        for j in range(PARTS_NUM):
            if (j != i):
                learningSet += parts[j]
                
        ns = 0
        n = len(learningSet)
        for email in learningSet:
            if (email.isSpam):
                ns += 1
                
        testingSet  = parts[i]
        
        
        spamDict, counters, unique = learnBayes(learningSet)
        res = testBayes2(spamDict, counters, testingSet, ns, n, unique, 0.01706422018348624)
        score = getF1Score(res, 0)

        if score > best_score:
            best_score = score
            best_result = spamDict, counters, testingSet, ns, n, unique
        print(score)
    
    return best_result


def getDiffs():
    diffs = []
    for i in range(PARTS_NUM):
        learningSet = []
        for j in range(PARTS_NUM):
            if (j != i):
                learningSet += parts[j]
                
        ns = 0
        n = len(learningSet)
        for email in learningSet:
            if (email.isSpam):
                ns += 1
                
        testingSet  = parts[i]
        
        
        spamDict, counters, unique = learnBayes(learningSet)
        res = testBayes2(spamDict, counters, testingSet, ns, n, unique, 0.01706422018348624)
        diffs += res
    
    return diffs

In [ ]:
diffs = getDiffs()

print(getF1Score(diffs, 0))

In [ ]:
d_l = min(diffs)[0] - 1
d_r = max(diffs)[0] + 1

diffsT = [x[0] for x in diffs if x[3]]         #spam
diffsF = [x[0] for x in diffs if not x[3]]     # ham

diffsT.sort(reverse=True)
diffsF.sort(reverse=True)

print(d_l)
print(d_r)
#print(diffsT)

npoints = []

x = d_r
curT = 0
curF = 0
while (x >= d_l):   # x = spam - ham  from max to min
    x -= 1
    
    #TP for spam
    while ((curT < len(diffsT)) and (diffsT[curT] > x)):
        curT += 1
    
    #FP for spam
    while ((curF < len(diffsF)) and (diffsF[curF] > x)):
        curF += 1
    
    npoints.append((curT/len(diffsT), curF/len(diffsF)))

 
points = []

for TPR, FPR in npoints:
    points += [np.array([TPR, FPR])]

points = np.array(points)

print(points)
plt.plot(points[:,1], points[:,0])

In [ ]:
diffs2 = getDiffs()

In [ ]:
d_l = min(diffs2)[0] - 1
d_r = max(diffs2)[0] + 1

diffsT = [x[0] for x in diffs2 if x[3]]         #spam
diffsF = [x[0] for x in diffs2 if not x[3]]     # ham

diffsT.sort(reverse=True)
diffsF.sort(reverse=True)

p_min = diffsT[-1]
n_max = diffsF[0]

tresh = None
if (p_min < n_max):
    print(p_min)
    tresh = p_min
else:
    print(n_max)
    tresh = n_max
    
print(getF1Score(diffs2, tresh))

print("diffs", d_l, d_r)
#print(diffsT)

npoints = []

x = d_r
curT = 0
curF = 0

fsc = 0
t = 0

while (x >= d_l):   # x = spam - ham  from max to min
    x -= 1
    
    #TP for spam
    while ((curT < len(diffsT)) and (diffsT[curT] > x)):
        curT += 1
    
    #FP for spam
    while ((curF < len(diffsF)) and (diffsF[curF] > x)):
        curF += 1
    
    
    # isCurrentSpam = d > diff
    #письма из спама, которые детектятся, как спам
    #письма из ок, которые детектятся, как спам
    #хотим занулить curF - ок письма, которые детектятся, как спам 
    f = getF1Score(diffs2, x)
    if ((curF < 3) and (f > fsc)):
        t = x
        fsc = f
    
    npoints.append((curT/len(diffsT), curF/len(diffsF)))


print("trash hold:", t, ", fscore:",  fsc)
#print("trash hold:", t, ", fscore:",  getF1Score(diffs2, 0))

points = []

for TPR, FPR in npoints:
    points += [np.array([TPR, FPR])]

points = np.array(points)

print(points)
plt.plot(points[:,1], points[:,0])